In [3]:
import os

In [9]:
%pwd

'/home/vaibhav/End_To_End_Animal_Classification_Using_MLFlow_DVC'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/home/vaibhav/End_To_End_Animal_Classification_Using_MLFlow_DVC'

In [7]:
#update entity module

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url : str 
    local_data_file : Path 
    unzip_dir: Path 

In [8]:
# Update the configuration manager in src config
from Animal_Classification.constants import *
from Animal_Classification.utils.common import read_yaml,create_directories

ModuleNotFoundError: No module named 'Animal_Classification'

In [ ]:



class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])       

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_url = config.source_url,
        local_data_file = config.local_data_file,
        unzip_dir=config.unzip_dir)

        return data_ingestion_config

In [ ]:
import os
import zipfile
import gdown
from Animal_Classification import logger
from Animal_Classification.utils.common import get_size
from Animal_Classification.entity.config_entity import DataIngestionConfig

In [ ]:
# update component
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
# update pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 



[2024-03-01 10:23:12,119: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-01 10:23:12,123: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-01 10:23:12,125: INFO: common: created directory at: artifacts]
[2024-03-01 10:23:12,126: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-01 10:23:12,127: INFO: 913165434: Downloading data from https://drive.google.com/file/d/1V5CB2o5Zzp7cXOAUt01px_QVlz7tpEB6/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1V5CB2o5Zzp7cXOAUt01px_QVlz7tpEB6
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1V5CB2o5Zzp7cXOAUt01px_QVlz7tpEB6&confirm=t&uuid=d4d7b6a6-9e2d-4780-a563-b517a1c39820
To: /home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:48<00:00, 1.18MB/s]


[2024-03-01 10:24:03,688: INFO: 913165434: Downloaded data from https://drive.google.com/file/d/1V5CB2o5Zzp7cXOAUt01px_QVlz7tpEB6/view?usp=sharing into file artifacts/data_ingestion/data.zip]
